## Bibliotecas

In [1]:
!pip install mip

import pandas as pd
import random
import math
import time
from mip import Model, xsum, maximize, BINARY, INTEGER, OptimizationStatus

Defaulting to user installation because normal site-packages is not writeable


# Leitura dos Dados

In [2]:
# Função para ler e processar os dados dos monitores e disciplinas a partir de um arquivo Excel.
# Ela lê o arquivo, limpa os nomes das colunas, remove duplicatas e extrai as colunas relevantes.
# Retorna um dicionário de alunos, uma lista de notas (Na) e uma matriz de disponibilidade (s_ad).
# Parâmetro:
# - path (str): Caminho para o arquivo Excel.
# Retornos:
# - alunos (dict): Dicionário onde a chave é o número USP e contem as seguintes informacoes --
#  -- > [Interesse em ser voluntario(ignorar), pontuacao do aluno (float), criterio de desempate (1, 2 ou 3), lista de materias que ele esta inscrito (Ex: [38, 39, 40]]).
# - Na (list): Lista das notas dos alunos.
# - s_ad (list of lists): Matriz binária de disponibilidade dos alunos para disciplinas.
def ler_dados(path):
    # Padroniza o nome das colunas, para facilitar extracao
    def limpar_nome_coluna(nome):
        return ' '.join(nome.strip().split()).lower()

    # Le e extrai as colunas necessarias
    df = pd.read_excel(path)
    df.columns = [limpar_nome_coluna(col) for col in df.columns]
    df = df.drop_duplicates(subset=['nºusp'], keep='last') # mantem apenas a ultima ocorrencia de um NUSP

    colunas_n = [
        'nºusp',
        'possui pedido de bolsa de estudos em andamento? (a concessão de bolsa de estudos implicará no cancelamento da monitoria a partir do início da vigência da bolsa)',
        'pretende se inscrever no peeg? (o acúmulo das duas monitorias não é permitido. caso o aluno seja selecionado nas duas modalidades precisará optar por uma delas)',
        'departamento de matemática (início da monitoria em 05/08/2024)',
        'departamento de matemática aplicada e estatística (início da monitoria em 05/08/2024)',
        'departamento de ciências de computação (início da monitoria em 05/08/2024)',
        'departamento de sistemas de computação (início da monitoria em 01/09/2024)',
        'tem interesse na monitoria voluntária (sem recebimento de bolsa)?',
        'média ponderada com reprovações'
    ]

    df_colunas = df[colunas_n].copy()

    # Calcula a pontuacao para criterio de desempate dos alunos (3 - mais prioridade, 2 - prioridade media, 1 - prioridade baixa)
    def calcular_pontuacao(row):
        bolsa = row[colunas_n[1]]
        peeg = row[colunas_n[2]]
        if bolsa == 'Sim' and peeg == 'Sim':
            return 1
        elif bolsa == 'Não' and peeg == 'Não':
            return 3
        return 2

    df_colunas['bolsa_peeg_status'] = df_colunas.apply(calcular_pontuacao, axis=1)
    df_colunas = df_colunas.drop([colunas_n[1], colunas_n[2]], axis=1)

    # Faz o mapeamento dos departamentos presentes na planilha
    departamento_mapping = {
        colunas_n[3]: 1,
        colunas_n[4]: 2,
        colunas_n[5]: 3,
        colunas_n[6]: 4
    }

    # Extrai as materias presentes na planilha
    materias_set = set()
    materias_dict = {}
    materia_index = 0

    for dept_col, dept_num in departamento_mapping.items():
        for materias in df_colunas[dept_col].dropna():
            for materia in materias.split(','):
                materia = materia.strip()
                if materia not in materias_set:
                    materias_set.add(materia)
                    materias_dict[materia_index] = (materia, dept_num)
                    materia_index += 1

    # Extrai os alunos, sua nota, sua pontuacao de desempate, se tem interesse em ser voluntario e as materias que ele se inscreveu
    alunos = df_colunas.set_index('nºusp').T.to_dict('list')

    Na = []
    for usp, valores in alunos.items():
        materias_combinadas = []
        valores_limpos = []
        for valor in valores:
            if isinstance(valor, str) and '-' in valor:
                for materia in valor.split(','):
                    materia = materia.strip()
                    for indice, (mat, dept) in materias_dict.items():
                        if materia == mat:
                            materias_combinadas.append(indice)
                            break
            elif isinstance(valor, str) and (valor.upper() == 'POS' or valor.upper() == 'PÓS'): # atribui a nota 1 caso o aluno for da pos, para diminuir sua prioridade
                valores_limpos.append(1)
            elif not pd.isna(valor):
                valores_limpos.append(valor)

        if materias_combinadas:
            valores_limpos.append(sorted(materias_combinadas))

        alunos[usp] = valores_limpos
        nota = valores_limpos[1] if len(valores_limpos) > 1 else None
        Na.append(nota)

    s_ad = [
        [1 if materia in valores_limpos[-1] else 0 for materia in materias_dict.keys()]
        for usp, valores_limpos in alunos.items()
    ]

    return alunos, Na, s_ad

# Modelo

In [3]:
# Função para criar o modelo de otimização de alocação de monitores com base nos dados fornecidos.
# Parâmetros:
# - alunos (dict): Dicionário de alunos (informações como notas, disponibilidade etc).
# - Na (list): Lista de notas dos alunos.
# - s_ad (list of lists): Matriz de disponibilidade dos alunos para disciplinas.
# - materias (list): Lista de todas as disciplinas.
# Retorno: Deve retornar o modelo de otimização, variáveis e os dados para análise posterior.
def criar_modelo(alunos, Na, s_ad, materias):
    A = list(range(len(alunos)))  # Conjunto de monitores
    D = list(range(len(materias)))  # Conjunto de disciplinas

    # Criação do modelo
    m = Model("Alocacao_de_Monitores", solver_name="CBC")

    # Variáveis de decisão
    x_ad = [[m.add_var(var_type=BINARY) for d in D] for a in A]  # x_ad[a][d]: 1 se monitor 'a' é alocado à disciplina 'd'
    y_d = [m.add_var(var_type=BINARY) for d in D]  # y_d[d]: 1 se disciplina 'd' não possui monitor
    # teste computacional 7
    # Extração do critério de desempate dos alunos
    desempate = [alunos[usp][2] if len(alunos[usp]) > 2 else 1 for usp in alunos]  # Padrão: 1 (baixa prioridade)
    # Definição do valor de Big M
    M = 1e-3  # Pequeno o suficiente para não alterar significativamente a solução
    #Função objetivo para teste computacional 7
    m.objective = maximize(
        xsum(Na[a] * x_ad[a][d] for a in A for d in D)  # Prioridade principal
        + M * xsum(desempate[a] * x_ad[a][d] for a in A for d in D)  # Peso secundário ajustado
        - xsum(y_d[d] for d in D)  # Penalidade para disciplinas sem monitor
    )
    # # Função objetivo
    # m.objective = maximize(
    #     xsum(Na[a] * x_ad[a][d] for a in A for d in D) - xsum(y_d[d] for d in D)
    # )

    # Restrições
    for d in D:
        m += xsum(x_ad[a][d] for a in A) + y_d[d] == 1  # Cada disciplina deve ter, no máximo, um monitor ou ficar sem monitor
    for a in A:
        m += xsum(x_ad[a][d] for d in D) <= 1  # Cada monitor pode ser alocado a, no máximo, uma disciplina
    for a in A:
        for d in D:
            m += x_ad[a][d] <= s_ad[a][d]  # Monitor só pode ser alocado a disciplinas que ele indicou interesse
    #teste computacional 3
    # monitores_disponiveis = A.copy()
    # disciplinas_disponiveis = D.copy()
    # alocacoes_fixadas = 0

    # for a in monitores_disponiveis.copy():
    #     disciplinas_compativeis = [d for d in disciplinas_disponiveis if s_ad[a][d] == 1]
    #     if disciplinas_compativeis:
    #         d = disciplinas_compativeis[0]  # Select the first compatible discipline
    #         x_ad[a][d].lb = 1  # Fix the variable to 1
    #         x_ad[a][d].ub = 1
    #         monitores_disponiveis.remove(a)
    #         disciplinas_disponiveis.remove(d)
    #         alocacoes_fixadas += 1
    #         if alocacoes_fixadas == 5:
    #             break
    return m, x_ad, y_d, D

# Função para resolver o modelo de otimização com base em parâmetros fornecidos.
# Parâmetros:
# - modelo (pulp.LpProblem): O modelo de otimização que será resolvido.
# - presolve (int): Se 1, presolve é ativado; se 0, é desativado.
# - cortes (int): Intensidade da geracao de cortes (0 desativa).
# Retorno: O modelo resolvido.
def resolver_modelo(modelo, presolve, cortes):
    # Configurar as opções do solver
    #teste computacional 6- comentar #modelo.preprocess = presolve 
    modelo.preprocess = presolve  # Define preprocess (-1, 0, 1)
    #teste computacional 5- comentar #modelo.cuts = cortes
    modelo.cuts = cortes  # Define geração de cortes (-1, 0, 1, 2, 3)  
    # Medir o tempo de execução
    start_time = time.time()

    # Resolver o modelo
    status = modelo.optimize(max_seconds=1800) # Limite de tempo de 30 minutos

    # Calcular o tempo de execução
    execution_time = time.time() - start_time

    # Exibir as informações do modelo resolvido
    print(f"Solução Encontrada: {modelo.objective_value}")
    print(f"Gap: {modelo.gap * 100:.2f}%")
    print(f"Tempo de execução: {execution_time:.2f} segundos")

    return modelo

# Função para exibir as informações dos monitores alocados e não alocados, além das disciplinas não atendidas.
# Parâmetros:
# - m (pulp.LpProblem): O modelo de otimização resolvido.
# - x_ad (dict): Variáveis de decisão indicando a alocação dos monitores às disciplinas.
# - y_d (dict): Variáveis de decisão indicando disciplinas não atendidas.
# - alunos (dict): Dicionário contendo as informações dos alunos.
# - D (list): Lista de disciplinas disponíveis.
# - s_ad (list of lists): Matriz de disponibilidade dos monitores para as disciplinas.
# Retorno: Nenhum. A função imprime os resultados diretamente.
def informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad):
    monitores_alocados = []
    monitores_nao_alocados = []

    # Exibe os monitores alocados para disciplinas
    for a in range(len(alunos)):
        alocado = False
        for d in D:
            if x_ad[a][d].x >= 0.99:  # Verifica se a variável é 1
                monitores_alocados.append(f"Monitor {list(alunos.keys())[a]} foi alocado para a disciplina {d}.")
                alocado = True

        # Caso o monitor não tenha sido alocado, armazenar a informação para exibir depois
        if not alocado:
            disciplinas_inscrito = [d for d in D if s_ad[a][d] == 1]
            monitores_nao_alocados.append(f"Monitor {list(alunos.keys())[a]} NÃO foi alocado. Inscrito para as disciplinas: {disciplinas_inscrito}")

    # Exibir os monitores alocados
    print("\nMonitores alocados:")
    for alocacao in monitores_alocados:
        print(alocacao)

    # Exibir os monitores não alocados
    print("\nMonitores não alocados:")
    for nao_alocacao in monitores_nao_alocados:
        print(nao_alocacao)

    # Mostrar disciplinas não atendidas e os monitores que se inscreveram para elas
    print("\nDisciplinas não atendidas e seus inscritos:")
    for d in D:
        if y_d[d].x >= 0.99:  # Verifica se a variável é 1
            monitores_inscritos = [list(alunos.keys())[a] for a in range(len(alunos)) if s_ad[a][d] == 1]
            if monitores_inscritos:
                print(f"A disciplina {d} não foi atendida por nenhum monitor. Inscritos: {monitores_inscritos}")
            else:
                print(f"A disciplina {d} não foi atendida por nenhum monitor e não possui inscritos.")


# Gerar Instâncias Maiores

In [4]:
# Função para aumentar os dados de alunos e matérias com base na escala fornecida.
# Parâmetros:
# - file_path (str): Caminho para o arquivo Excel com os dados originais.
# - escala (float): Fator de aumento dos dados (e.g., 1.5 para aumentar em 50%).
# Retorno: Retorna os dados aumentados (alunos, notas, disponibilidade dos alunos para disciplinas e a lista de matérias).
def aumentar_dados(path, escala):
    # Lê os dados originais usando a função existente
    alunos_originais, Na_originais, s_ad_originais = ler_dados(path)
    seed = 0

    # Se seed for 0 e escala for 1, retornar os dados originais
    if seed == 0 and escala == 1:
        todas_materias = list(range(len(s_ad_originais[0])))
        return alunos_originais, Na_originais, s_ad_originais, todas_materias

    # Define a seed para a geração de dados aleatórios
    if seed is not None:
        random.seed(seed)

    novos_alunos = {}
    novas_notas = []
    nova_s_ad = []
    usp_existentes = set(alunos_originais.keys())
    max_usp = max(usp_existentes)

    # Aumenta o número de materias em 1.12x com base na escala
    num_materias_originais = len(s_ad_originais[0])
    num_novas_materias = math.ceil(num_materias_originais * 1.12 * escala)

    # Novos índices para as matérias que serão adicionadas
    novas_materias = list(range(num_materias_originais, num_materias_originais + num_novas_materias))

    # Aumenta o numero de alunos conforme a escala
    total_alunos = len(alunos_originais)
    num_novos_alunos = math.ceil(total_alunos * (escala - 1))  # Cria novos alunos além dos originais

    # Cria os novos alunos
    for i in range(num_novos_alunos):
        # Novo Nusp
        max_usp += 1
        novo_usp = max_usp
        usp_existentes.add(novo_usp)

        # Gerar uma nova nota aleatória para o novo aluno
        nova_nota = random.uniform(0, 10)
        novas_notas.append(nova_nota)

        # Gerar matérias aleatórias (entre 2 e 4) para cada novo aluno
        num_materias_aluno = random.randint(2, 4)
        todas_materias = list(range(num_materias_originais)) + novas_materias  # Considera matérias antigas e novas
        materias_aluno = random.sample(todas_materias, num_materias_aluno)

        # Criar a matriz de disponibilidade (s_ad) para o novo aluno com as matérias selecionadas
        disponibilidade_aluno = [1 if materia in materias_aluno else 0 for materia in todas_materias]
        nova_s_ad.append(disponibilidade_aluno)

        novos_alunos[novo_usp] = []

    # Expande a matriz s_ad original para acomodar as novas matérias
    s_ad_expandidos = [row + [0] * len(novas_materias) for row in s_ad_originais]
    s_ad_expandidos += nova_s_ad

    # Combinar os dados originais e os novos
    alunos_expandidos = {**alunos_originais, **novos_alunos}
    Na_expandidos = Na_originais + novas_notas

    return alunos_expandidos, Na_expandidos, s_ad_expandidos, todas_materias

# Exemplo de Uso

In [5]:
# Caminho do arquivo Excel com os dados originais
path = 'Dados_monitores.xlsx'

# Aumenta os dados de alunos e matérias
#alunos, Na, s_ad, materias = aumentar_dados(path, escala=1)

# teste computacional 4,5, 6 e 7
#alunos, Na, s_ad, materias = aumentar_dados(path, escala=2)
#alunos, Na, s_ad, materias = aumentar_dados(path, escala=3)
#alunos, Na, s_ad, materias = aumentar_dados(path, escala=5)
#alunos, Na, s_ad, materias = aumentar_dados(path, escala=10)
alunos, Na, s_ad, materias = aumentar_dados(path, escala=15)
# Cria o modelo de otimização com os dados aumentados
mod, x_ad, y_d, D = criar_modelo(alunos, Na, s_ad, materias)

# Resolve o modelo
m = resolver_modelo(mod, 1, 1)

informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad)

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 2375 (-1632712) rows, 5419 (-1628213) columns and 9758 (-4888894) elements
Clp0030I 21 infeas 0.02822934, obj 6602.058 - mu 6.1277099e-05, its 105, 3296 interior
Clp0030I 45 infeas 5.5692825e-06, obj 6601.9985 - mu 9.3321284e-09, its 105, 4004 interior
Clp1000I sum of infeasibilities 0.000165762 - average 6.97945e-08, 1511 fixed columns
Coin0506I Presolve 2057 (-318) rows, 3694 (-1725) columns and 7216 (-2542) elements
Clp0006I 0  Obj 6086.1908 Primal inf 0.00011783124 (3) Dual inf 9.0000019e+12 (3619)
Clp0029I End of values pass after 3619 iterations
Clp0014I Perturbing problem by 0.001% of 1 - largest nonzero change 2.9936918e-05 ( 0.0014968459%) - largest zero change 2.9976734e-05
Clp0000I Optimal - objective value 6294.1
Clp0000I Optimal - objective value 6294.1
Coin0511I After Postsolve, objective 6294.1, i